In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import src

In [ ]:
dfs = []
for fold in range(1, 6):
    df = pd.read_csv(src.PATH / f"results/kfold/fold_{fold}.csv")
    df["fold"] = fold
    dfs.append(df)

In [ ]:
df = pd.concat(dfs)
df.rename({"Unnamed: 0": "measure"}, axis=1, inplace=True)

In [ ]:
rows = []
for feat in ["elite", "centr", "left", "right", "micro avg", "macro avg"]:
    for measure in ["precision", "recall", "f1-score"]:
        row = dict()
        values = df.loc[df.measure == measure, feat]
        row["name"] = feat
        row["measure"] = measure
        row["value"] = round(np.mean(values), 3)
        row["std"] = round(np.std(values), 3)
        row["cell"] = f"{row['value']} ({row['std']})"
        rows.append(row)
results = pd.DataFrame(rows)

In [ ]:
values = results.pivot(index="name", columns="measure", values="cell").reset_index()
order_by = CategoricalDtype(
    ["elite", "centr", "left", "right", "micro avg", "macro avg"], ordered=True
)
values["name"] = values.name.astype(order_by)
values = values.loc[:, ["name", "precision", "recall", "f1-score"]].sort_values("name")
values = values.set_index("name", drop=True)

In [ ]:
values

measure,precision,recall,f1-score
name,,,
elite,0.798 (0.032),0.901 (0.029),0.845 (0.011)
centr,0.658 (0.013),0.751 (0.052),0.7 (0.018)
left,0.704 (0.035),0.729 (0.053),0.714 (0.019)
right,0.596 (0.04),0.746 (0.048),0.66 (0.019)
micro avg,0.723 (0.026),0.815 (0.032),0.765 (0.008)
macro avg,0.689 (0.026),0.782 (0.036),0.73 (0.012)


In [ ]:
style = values.style.set_table_styles(
    [
        {"selector": "toprule", "props": ":toprule;"},
        {"selector": "bottomrule", "props": ":bottomrule;"},
    ]
).format(thousands=",")

latex = style.to_latex(
    multirow_align="t",
    # clines="skip-last;index",
)

In [ ]:
print(latex)

\begin{tabular}{llll}
\toprule
measure & precision & recall & f1-score \\
name &  &  &  \\
elite & 0.798 (0.032) & 0.901 (0.029) & 0.845 (0.011) \\
centr & 0.658 (0.013) & 0.751 (0.052) & 0.7 (0.018) \\
left & 0.704 (0.035) & 0.729 (0.053) & 0.714 (0.019) \\
right & 0.596 (0.04) & 0.746 (0.048) & 0.66 (0.019) \\
micro avg & 0.723 (0.026) & 0.815 (0.032) & 0.765 (0.008) \\
macro avg & 0.689 (0.026) & 0.782 (0.036) & 0.73 (0.012) \\
\bottomrule
\end{tabular}



In [ ]:
out_path = src.PATH / "tmp"
_ = (out_path / "kfold_results.tex").write_text(latex)